In [49]:
import cv2
import numpy as np

def find_template_count(template_path, target_path, threshold=0.8):
    # Load the template and target images
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    target = cv2.imread(target_path, cv2.IMREAD_COLOR)
    
    # Convert images to grayscale
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    
    # Get dimensions of the template image
    w, h = template_gray.shape[::-1]
    
    # Perform template matching
    result = cv2.matchTemplate(target_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)
    
    # Initialize coordinates list
    coordinates = []
    
    # Store all coordinates where match is found
    for pt in zip(*locations[::-1]):
        coordinates.append((pt[0] + w // 2, pt[1] + h // 2))  # Get the center of the match
    
    # Remove overlapping matches
    coordinates = remove_overlapping_matches(coordinates, w, h)

    # Return the count of matches
    return len(coordinates)



In [68]:
# Paths to the template and target images
template_path =  "C:\\macro\\images\\reduser.png"
target_path =  "C:\\macro\\images\\red_result.png"

# Find the number of red user instances
red_user_count = find_template_count(template_path, target_path)

print("Number of red users found:", red_user_count)

Number of red users found: 1


In [52]:
# Paths to the template and target images
template_path =  "C:\\macro\\images\\user.png"
target_path =  "C:\\macro\\images\\yellow_result.png"

# Find the number of red user instances
red_user_count = find_template_count(template_path, target_path)

print("Number of red users found:", red_user_count)

Number of red users found: 1


In [45]:
# Load the template and target images
template = cv2.imread(template_path, cv2.IMREAD_COLOR)
target = cv2.imread(target_path, cv2.IMREAD_COLOR)

# Convert images to grayscale
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

# Get dimensions of the template image
w, h = template_gray.shape[::-1]

# Perform template matching
result = cv2.matchTemplate(target_gray, template_gray, cv2.TM_CCOEFF_NORMED)


In [46]:
np.max(result)

0.99999976

In [53]:
import cv2
import numpy as np

def find_template_coordinates(template_path, target_path, threshold=0.8):
    # Load the template and target images
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    target = cv2.imread(target_path, cv2.IMREAD_COLOR)
    
    # Convert images to grayscale
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    
    # Get dimensions of the template image
    w, h = template_gray.shape[::-1]
    
    # Perform template matching
    result = cv2.matchTemplate(target_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)
    
    # Initialize coordinates list
    coordinates = []
    
    # Store all coordinates where match is found
    for pt in zip(*locations[::-1]):
        coordinates.append((pt[0] + w // 2, pt[1] + h // 2))  # Get the center of the match
    
    # Remove overlapping matches
    coordinates = remove_overlapping_matches(coordinates, w, h)

    # Return the coordinates of matches
    return coordinates

def remove_overlapping_matches(coordinates, w, h):
    filtered_coords = []
    for coord in coordinates:
        if not any([np.linalg.norm(np.array(coord) - np.array(existing)) < min(w, h) // 2 for existing in filtered_coords]):
            filtered_coords.append(coord)
    return filtered_coords

# Paths to the template and target images
template_path =  "C:\\macro\\images\\user.png"
target_path =  "C:\\macro\\images\\yellow_result.png"

# Find the coordinates of yellow user instances
yellow_user_coordinates = find_template_coordinates(template_path, target_path)

print("Coordinates of yellow users found:", yellow_user_coordinates)


Coordinates of yellow users found: [(226, 80)]


In [82]:
import pyautogui
import cv2
import os
import numpy as np
import common as cm

# Define the color ranges for yellow and red dots
yellow_lower = np.array([20, 100, 100], np.uint8)
yellow_upper = np.array([30, 255, 255], np.uint8)

red_lower = np.array([0, 100, 100], np.uint8)
red_upper = np.array([10, 255, 255], np.uint8)
base_path = "C:\\macro\\images"

def make_masked_map():
    img_path =  "C:\\macro\\images\\screenshot.png"
    screen_image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    hsv_screenshot = cv2.cvtColor(screen_image, cv2.COLOR_BGR2HSV)

    # Find the yellow dot
    yellow_mask = cv2.inRange(hsv_screenshot, yellow_lower, yellow_upper)

    # Find the red dots
    red_mask = cv2.inRange(hsv_screenshot, red_lower, red_upper)

    # Save the results for debugging
    yellow_result = cv2.bitwise_and(screen_image, screen_image, mask=yellow_mask)
    red_result = cv2.bitwise_and(screen_image, screen_image, mask=red_mask)
    cv2.imwrite(os.path.join(base_path, 'yellow_result.png'), yellow_result)
    cv2.imwrite(os.path.join(base_path, 'red_result.png'), red_result)
    print("마스크 캡쳐 완료")

    return yellow_result, red_result

def remove_overlapping_matches(coordinates, w, h):
    filtered_coords = []
    for coord in coordinates:
        if not any([np.linalg.norm(np.array(coord) - np.array(existing)) < min(w, h) // 2 for existing in filtered_coords]):
            filtered_coords.append(coord)
    return filtered_coords


def find_match(template, target_name,threshold=0.8):
    # Load the template and target images
    
    target_path = os.path.join(base_path, "{}.png".format(target_name))
    target = cv2.imread(target_path, cv2.IMREAD_COLOR)
    
    # Convert images to grayscale
    template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
    target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    
    # Get dimensions of the template image
    w, h = template_gray.shape[::-1]
    
    # Perform template matching
    result = cv2.matchTemplate(target_gray, template_gray, cv2.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)
    
    # Initialize coordinates list
    coordinates = []
    
    # Store all coordinates where match is found
    for pt in zip(*locations[::-1]):
        coordinates.append((pt[0] + w // 2, pt[1] + h // 2))  # Get the center of the match
    
    # Remove overlapping matches
    coordinates = remove_overlapping_matches(coordinates, w, h)

    # Return the count of matches
    return len(coordinates), coordinates

def find_user():
    yellow_result, red_result = make_masked_map()
    red_user_cnt, _ = find_match(red_result, 'reduser')
    _, user = find_match(yellow_result, 'user')

    print("타 유저 수: ", red_user_cnt)
    print("유저 위치: ", user)

In [83]:
yellow_result, red_result = make_masked_map()

마스크 캡쳐 완료


In [84]:
find_user()

마스크 캡쳐 완료
타 유저 수:  1
유저 위치:  [(383, 131)]
